# Random Forest Classification
The Random Forest algorithm is a classification algorithm which builds several decision trees, and aggregates each of their outputs to make a prediction. This makes it more robust to overfitting.

In order to convert your dataset to cudf format please read the cudf documentation on https://rapidsai.github.io/projects/cudf/en/latest/. For additional information on the RandomForest model please refer to the documentation on https://rapidsai.github.io/projects/cuml/en/latest/index.html

This notebook demonstratrates fitting a RandomForestClassifier on the Higgs dataset. It is a binary classification problem to distinguish between a signal process which produces Higgs bosons and a background process which does not. The notebook also compares the performance (accuracy and speed) with sklearn's parallel RandomForestClassifier implementation.

In [1]:
from cuml import RandomForestClassifier as cuRF
from sklearn.ensemble import RandomForestClassifier as sklRF
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import cudf
import numpy as np
import pandas as pd
import os
from urllib.request import urlretrieve
import gzip

# Helper function to download and extract the Higgs dataset

In [2]:
def download_higgs(compressed_filepath, decompressed_filepath):
    higgs_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz'
    if not os.path.isfile(compressed_filepath):
        urlretrieve(higgs_url, compressed_filepath)
    if not os.path.isfile(decompressed_filepath):
        cf = gzip.GzipFile(compressed_filepath)
        with open(decompressed_filepath, 'wb') as df:
            df.write(cf.read())

# Download Higgs data and read using cudf

In [3]:
compressed_filepath = 'HIGGS.csv.gz' # Set this as path for gzipped Higgs data file, if you already have
decompressed_filepath = 'HIGGS.csv' # Set this as path for decompressed Higgs data file, if you already have
download_higgs(compressed_filepath, decompressed_filepath)

col_names = ['label'] + ["col-{}".format(i) for i in range(2, 30)] # Assign column names
dtypes_ls = ['int32'] + ['float32' for _ in range(2, 30)] # Assign dtypes to each column
data = cudf.read_csv(decompressed_filepath, names=col_names, dtype=dtypes_ls)
data.head().to_pandas()

,label,col-2,col-3,col-4,col-5,col-6,col-7,col-8,col-9,col-10,...,col-20,col-21,col-22,col-23,col-24,col-25,col-26,col-27,col-28,col-29
0,1,0.869293,-0.635082,0.225690,0.327470,-0.689993,0.754202,-0.248573,-1.092064,0.000000,...,-0.010455,-0.045767,3.101961,1.353760,0.979563,0.978076,0.920005,0.721657,0.988751,0.876678
1,1,0.907542,0.329147,0.359412,1.497970,-0.313010,1.095531,-0.557525,-1.588230,2.173076,...,-1.138930,-0.000819,0.000000,0.302220,0.833048,0.985700,0.978098,0.779732,0.992356,0.798343
2,1,0.798835,1.470639,-1.635975,0.453773,0.425629,1.104875,1.282323,1.381664,0.000000,...,1.128848,0.900461,0.000000,0.909753,1.108331,0.985692,0.951331,0.803251,0.865924,0.780118
3,0,1.344385,-0.876626,0.935913,1.992050,0.882454,1.786066,-1.646778,-0.942383,0.000000,...,-0.678379,-1.360356,0.000000,0.946652,1.028704,0.998656,0.728281,0.869200,1.026736,0.957904
4,1,1.105009,0.321356,1.522401,0.882808,-1.205349,0.681466,-1.070464,-0.921871,0.000000,...,-0.373566,0.113041,0.000000,0.755856,1.361057,0.986610,0.838085,1.133295,0.872245,0.808486


# Make train test splits

In [4]:
X, y = data[data.columns.difference(['label'])].as_matrix(), data['label'].to_array() # Separate data into X and y
del data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=500_000)

In [5]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(10500000, 28) (10500000,) (500000, 28) (500000,)


#### You can consult RandomForestClassifier docstring to check all the parameters, but here are some of the more important ones: 
1. n_estimators: (default = 10) number of trees in the forest.
2. max_depth: (default = -1) Maximum tree depth. Unlimited (i.e, until leaves are pure), if -1.
3. n_bins: (default = 8) Number of bins used by the split algorithm.

Note on `nbins`: Reducing `n_bins` shrinks the histograms used to compute which tree nodes to split. This reduction improves training time, but if you reduce it too low, you may harm model accuracy. 

In [6]:
# cuml Random Forest params

cu_rf_params = {
    'n_estimators': 25,
    'max_depth': 13,
    'n_bins': 15,
}

#### The methods that can be used with the RandomForestClassifier are:
1. fit: Fit the model with X and y.
2. get_params: Sklearn style return parameter state
3. predict: Predicts the y for X.
4. set_params: Sklearn style set parameter state to dictionary of params.
5. cross_validate: Predicts the accuracy of the model for X.

###### Note on input to `fit` method: Since `fit` is processed on the GPU, it can accept `cudf` dataframes or `numpy` arrays

In [7]:
%%time
# Train cuml RF

cu_rf = cuRF(**cu_rf_params)
cu_rf.fit(X_train, y_train)

CPU times: user 40.4 s, sys: 13.1 s, total: 53.5 s
Wall time: 53.6 s


#### Set Sklearn params and fit RandomForestClassifier

In [8]:
# sklearn Random Forest params

skl_rf_params = {
    'n_estimators': 25,
    'max_depth': 13,
}

In [9]:
%%time
# Train sklearn RF parallely

skl_rf = sklRF(**skl_rf_params, n_jobs=20)
skl_rf.fit(X_train, y_train)

CPU times: user 56min 27s, sys: 1min, total: 57min 27s
Wall time: 4min 30s


# Predict and compare cuml and sklearn RandomForestClassifier

###### Note on input to cuml `predict` method: Since `predict` is processed on the CPU, it can only accept `numpy` arrays

In [10]:
# Predict

print("cuml RF Accuracy Score: ", accuracy_score(cu_rf.predict(X_test), y_test))
print("sklearn RF Accuracy Score: ", accuracy_score(skl_rf.predict(X_test), y_test))

cuml RF Accuracy Score:  0.72466
sklearn RF Accuracy Score:  0.72177
